In [1]:
import autograd.numpy as np
import matplotlib.pyplot as plt
from autograd import grad, elementwise_grad, hessian, jacobian

import sys
sys.path.insert(0, "../../project_2/src")
from SGD import minibatch

In [14]:
# Inspired by mortens example code in week 43

def activation_function(z):
    return np.maximum(0, z)

def activation_out(z):
    return z

"""
network_shape = [inputs, w1, w2, ..., wL, outputs]

W = wwwwb
    wwwwb
"""
def initialize_params(network_shape):
    P = []
    for i in range(1, len(network_shape)):
        k = network_shape[i-1]
        j = network_shape[i]
        P.append(np.random.randn(j, k) * np.sqrt(2) / np.sqrt(k))
        P[i-1] = np.concatenate((P[i-1], np.zeros(j).reshape(-1,1)), axis=1)

    return P

def Network(t, P):
    # Assumes the input t to be a scalar, returns a 2d-row vector e.g. shape=[1,6]
    a = t.reshape(-1,1)
    for P_i in P:
        #a = np.concatenate((a, np.ones(np.size(a, 0)).reshape(-1, 1)), axis=1) 
        a = np.concatenate((a, np.ones((1,1))), axis=1) 
        z = np.matmul(a, np.transpose(P_i))
        a = activation_function(z)
    
    return activation_out(z) 

def Network_predict(t, P):
    # For predictions
    # Assumes the input t to be a 1d-array, 
    # returns a matrix where each row corresponds to a vector for a particular t
    a = t.reshape(-1,1)
    for P_i in P:
        a = np.concatenate((a, np.ones(np.size(a, 0)).reshape(-1, 1)), axis=1) 
        #a = np.concatenate((a, np.ones((1,1))), axis=1) 
        z = np.matmul(a, np.transpose(P_i))
        a = activation_function(z)
    
    return activation_out(z) 

def optimize(t, P, A, x_0, N_minibatches, learning_rate, n_epochs):
    # Assumes t is a 1d-array.
    assert N_minibatches <= np.size(t, 0)

    cost_func_grad = grad(costfunction, 1) # Check which grad-call is correct.
    
    for epoch in range(n_epochs):
        mb = minibatch(t, N_minibatches)
        for i in range(N_minibatches):
            t_mb = t[mb[i]]
            M = np.size(t_mb, 0)
            # compute gradients of weights
            cost_grad = cost_func_grad(t_mb, P, A, x_0)
            for l in range(len(P)):
                P[l] -= learning_rate * cost_grad[l]
    return P

def g_trial(t, P, x_0):
    #assumes the input t to be a scalar, x_0 is 1d-row-vector which broadcasts to e.g. (1,6)
    
    return np.exp(-10*t)*x_0 + (1-np.exp(- 10*t))*Network(t,P)

def g_trial_predict(t, P, x_0):
    # For predictions
    #assumes the input t to be a 1d-array, broadcasts along the rows of Network_predict's output
    t = t.reshape(-1,1)
    return np.exp(-t*10)*x_0 + (1-np.exp(-t*10))*Network_predict(t,P)

def costfunction(t, P, A, x_0):
    
    cost = 0
    g_grad = jacobian(g_trial,0) # Check that this is the correct grad-call
    
    for time in t:
        d_dt = g_grad(time,P,x_0).reshape(-1,1) # should have shape (eigenvector_length,1)
        x_t = g_trial(time,P,x_0).reshape(-1,1) #check shape, should be (eigenvector_length,1)
        # Right hand side is: -x + [x.T @ x * A + (1 - x.T @ A @ x) @ I] @ x where x is x_t is a column vector
        right_side = (-x_t + (x_t.T @ x_t * A + (1 - x_t.T @ A @ x_t) * np.identity(np.size(x_t,0))) @ x_t)
        print(x_t)
        print(right_side)
#         right_side = -x_t + np.matmul( # -x + [
#                                 np.matmul(np.transpose(x_t), x_t) * A #(x.T @ x) * A
#                                 + ( 1 - np.matmul(np.transpose(x_t),np.matmul(A, x_t))) # + (1-x.t @ A @ x)
#                                 * np.identity(np.size(x_t, 0)) # * I
#                                 , x_t) # ] @ x
        cost = cost + np.sum((d_dt - right_side)**2) / np.size(x_t) 
                                                              
        

    return cost / np.size(t)

In [15]:
t = np.linspace(0,1,100)
Q = np.random.randn(6,6)
A = (np.transpose(Q) + Q) /2 #standard trick for making symmetric real
A = np.array([[0.7663, 0.4283, -0.3237, -0.4298, -0.1438], [0.4283,0.2862,0.0118,-0.2802,0.1230]
              , [-0.3237,0.0118,-0.9093,-0.4383,0.7684],[-0.4298,-0.2802,-0.4384,-0.0386,-0.1315]
              , [-0.1438,0.1230,0.7684,-0.1315,-0.4480]]) #matrix from Yi et al.

x_0 = np.random.randn(np.size(A,0)) #initial guess for eigenvector.

network_shape = [1, 10, np.size(x_0)] #output must match eigenvector length
P = initialize_params(network_shape)

In [16]:
P = optimize(t, P, A, x_0,10, 0.002, 10)
#y = Network(X[0, :].reshape(1, 2), P)

Autograd ArrayBox with value [[0.10776322]
 [0.5390781 ]
 [0.35104005]
 [0.16099903]
 [0.15814269]]
Autograd ArrayBox with value [[ 0.05364721]
 [ 0.09647915]
 [-0.13405239]
 [-0.1768419 ]
 [ 0.1121654 ]]
Autograd ArrayBox with value [[ 1.45077298]
 [-0.66823307]
 [-2.04405343]
 [ 0.93416958]
 [-0.3303678 ]]
Autograd ArrayBox with value [[  7.06912588]
 [  1.56776241]
 [  7.88728289]
 [  2.52787303]
 [-13.78009421]]
Autograd ArrayBox with value [[ 0.31965421]
 [-0.03030808]
 [-0.35049708]
 [ 0.22402379]
 [-0.03650811]]
Autograd ArrayBox with value [[ 0.04969859]
 [ 0.01775028]
 [ 0.04749419]
 [-0.00883392]
 [-0.0897662 ]]
Autograd ArrayBox with value [[ 1.30215942]
 [-0.59956776]
 [-1.8344836 ]
 [ 0.83850875]
 [-0.2964596 ]]
Autograd ArrayBox with value [[ 5.11065564]
 [ 1.13379139]
 [ 5.70185144]
 [ 1.8264764 ]
 [-9.9620081 ]]
Autograd ArrayBox with value [[0.11282426]
 [0.48018416]
 [0.29559414]
 [0.15545623]
 [0.13943834]]
Autograd ArrayBox with value [[ 0.04245408]
 [ 0.06708769]
 

 [-0.5118857 ]]
Autograd ArrayBox with value [[ 0.49483146]
 [ 0.39178018]
 [ 0.62651781]
 [-0.70279302]
 [-1.77612843]]
Autograd ArrayBox with value [[ 0.634311  ]
 [-0.21693832]
 [-0.6155806 ]
 [ 0.64720644]
 [-0.37154785]]
Autograd ArrayBox with value [[ 0.11943145]
 [ 0.09910151]
 [ 0.07736414]
 [-0.28437174]
 [-0.4774979 ]]
Autograd ArrayBox with value [[ 0.34020081]
 [-0.07038964]
 [-0.26379756]
 [ 0.40560224]
 [-0.22847095]]
Autograd ArrayBox with value [[ 0.01330273]
 [ 0.00408343]
 [-0.04659099]
 [-0.06735525]
 [-0.04723   ]]
Autograd ArrayBox with value [[ 0.09453846]
 [ 0.15708495]
 [ 0.10917998]
 [ 0.20422343]
 [-0.06151791]]
Autograd ArrayBox with value [[ 0.00757652]
 [ 0.01073703]
 [-0.01802997]
 [-0.00068131]
 [ 0.00479937]]
Autograd ArrayBox with value [[ 0.45087516]
 [-0.12959408]
 [-0.39923096]
 [ 0.49650199]
 [-0.28414351]]
Autograd ArrayBox with value [[ 0.03348109]
 [ 0.02440273]
 [-0.03103946]
 [-0.12985349]
 [-0.1412922 ]]
Autograd ArrayBox with value [[ 0.57331

Autograd ArrayBox with value [[ 0.28261198]
 [-0.01408507]
 [-0.15646293]
 [ 0.38020817]
 [-0.21335337]]
Autograd ArrayBox with value [[ 0.01189937]
 [-0.00389577]
 [-0.06812054]
 [-0.045377  ]
 [-0.01488887]]
Autograd ArrayBox with value [[ 0.65368275]
 [-0.19372864]
 [-0.58679343]
 [ 0.69179868]
 [-0.40661977]]
Autograd ArrayBox with value [[ 0.10882604]
 [ 0.08851772]
 [-0.0464109 ]
 [-0.36898916]
 [-0.42802472]]
Autograd ArrayBox with value [[ 0.59596175]
 [-0.16813068]
 [-0.52158437]
 [ 0.64337234]
 [-0.37766796]]
Autograd ArrayBox with value [[ 0.07812786]
 [ 0.06003354]
 [-0.07184128]
 [-0.29526771]
 [-0.30722239]]
Autograd ArrayBox with value [[ 0.45703035]
 [-0.10477055]
 [-0.36334128]
 [ 0.52678117]
 [-0.30715488]]
Autograd ArrayBox with value [[ 0.03175554]
 [ 0.01535267]
 [-0.09350818]
 [-0.15531821]
 [-0.11374909]]
Autograd ArrayBox with value [[ 0.6190594 ]
 [-0.17840409]
 [-0.54770066]
 [ 0.66275118]
 [-0.38926759]]
Autograd ArrayBox with value [[ 0.08949901]
 [ 0.070658

 [-0.07010907]]
Autograd ArrayBox with value [[ 0.00780434]
 [ 0.02032383]
 [-0.02431814]
 [ 0.0015385 ]
 [ 0.01078839]]
Autograd ArrayBox with value [[ 0.29864021]
 [ 0.00638197]
 [-0.14072759]
 [ 0.41272043]
 [-0.22941922]]
Autograd ArrayBox with value [[ 0.01827616]
 [-0.00579329]
 [-0.09601586]
 [-0.05364443]
 [-0.013979  ]]
Autograd ArrayBox with value [[ 0.41947451]
 [-0.05267222]
 [-0.27951389]
 [ 0.51615271]
 [-0.29676656]]
Autograd ArrayBox with value [[ 0.02984391]
 [ 0.00025885]
 [-0.13998299]
 [-0.13600172]
 [-0.06255873]]
Autograd ArrayBox with value [[ 0.56177912]
 [-0.11550204]
 [-0.43809621]
 [ 0.63770592]
 [-0.37277586]]
Autograd ArrayBox with value [[ 0.06364288]
 [ 0.02958655]
 [-0.17229038]
 [-0.28592736]
 [-0.19991067]]
Autograd ArrayBox with value [[0.03130457]
 [0.51736698]
 [0.4522617 ]
 [0.17245769]
 [0.13134075]]
Autograd ArrayBox with value [[ 0.00726183]
 [ 0.13700661]
 [-0.14429859]
 [-0.17566956]
 [ 0.18612857]]
Autograd ArrayBox with value [[ 0.7514171 ]


Autograd ArrayBox with value [[ 0.40610979]
 [-0.01489352]
 [-0.23231763]
 [ 0.5193899 ]
 [-0.28493861]]
Autograd ArrayBox with value [[ 0.035439  ]
 [-0.00494519]
 [-0.16498646]
 [-0.12799698]
 [-0.04802895]]
Autograd ArrayBox with value [[ 0.68897292]
 [-0.13281241]
 [-0.54099025]
 [ 0.76374325]
 [-0.43692608]]
Autograd ArrayBox with value [[ 0.12665434]
 [ 0.06344585]
 [-0.26186334]
 [-0.50799725]
 [-0.38331183]]
Autograd ArrayBox with value [[ 0.54252505]
 [-0.07274777]
 [-0.38188455]
 [ 0.63728655]
 [-0.35873012]]
Autograd ArrayBox with value [[ 0.06469626]
 [ 0.01355624]
 [-0.22490146]
 [-0.27554185]
 [-0.15498995]]
Autograd ArrayBox with value [[ 0.36402504]
 [ 0.00391279]
 [-0.18549091]
 [ 0.48296641]
 [-0.26169445]]
Autograd ArrayBox with value [[ 0.03020301]
 [-0.00647478]
 [-0.14422871]
 [-0.09442542]
 [-0.03011862]]
Autograd ArrayBox with value [[0.01520497]
 [0.38474997]
 [0.36339788]
 [0.16889992]
 [0.04347622]]
Autograd ArrayBox with value [[-0.00408372]
 [ 0.0787447 ]
 

...

Autograd ArrayBox with value [[0.02691994]
 [0.60609109]
 [0.54563792]
 [0.19538195]
 [0.17509978]]
Autograd ArrayBox with value [[ 0.00048465]
 [ 0.21910661]
 [-0.23708876]
 [-0.29828035]
 [ 0.31314491]]
Autograd ArrayBox with value [[0.00933888]
 [0.54312764]
 [0.50400441]
 [0.19383835]
 [0.11349556]]
Autograd ArrayBox with value [[-0.01196734]
 [ 0.18097009]
 [-0.17302049]
 [-0.20172428]
 [ 0.24782374]]
Autograd ArrayBox with value [[ 0.13550866]
 [ 0.40347346]
 [ 0.30563998]
 [ 0.33020612]
 [-0.01311003]]
Autograd ArrayBox with value [[ 0.04332904]
 [ 0.11780731]
 [-0.11595864]
 [-0.0510416 ]
 [ 0.0844955 ]]
Autograd ArrayBox with value [[ 0.34865625]
 [ 0.35020602]
 [ 0.09227899]
 [ 0.52921221]
 [-0.12653355]]
Autograd ArrayBox with value [[ 0.13842129]
 [ 0.08853029]
 [-0.27559682]
 [-0.09857241]
 [ 0.01318044]]
Autograd ArrayBox with value [[0.04346837]
 [0.64393015]
 [0.55925878]
 [0.19359092]
 [0.19571174]]
Autograd ArrayBox with value [[ 0.02004402]
 [ 0.24184985]
 [-0.277212

Autograd ArrayBox with value [[ 0.33602765]
 [ 0.38381177]
 [ 0.12319675]
 [ 0.51344716]
 [-0.10251785]]
Autograd ArrayBox with value [[ 0.14203491]
 [ 0.10598054]
 [-0.26920784]
 [-0.10244613]
 [ 0.02573239]]
Autograd ArrayBox with value [[0.03696182]
 [0.63349539]
 [0.56030987]
 [0.19477243]
 [0.19286607]]
Autograd ArrayBox with value [[ 0.0113027 ]
 [ 0.23792375]
 [-0.26784444]
 [-0.34257962]
 [ 0.34044131]]
Autograd ArrayBox with value [[ 0.30814887]
 [ 0.38888436]
 [ 0.15004248]
 [ 0.48771924]
 [-0.08870985]]
Autograd ArrayBox with value [[ 0.1278538 ]
 [ 0.1094022 ]
 [-0.24131478]
 [-0.08739755]
 [ 0.0350567 ]]
Autograd ArrayBox with value [[0.0753056 ]
 [0.45715481]
 [0.39154933]
 [0.2683352 ]
 [0.04082686]]
Autograd ArrayBox with value [[ 0.01879557]
 [ 0.1422746 ]
 [-0.11875214]
 [-0.09539846]
 [ 0.13812364]]
Autograd ArrayBox with value [[0.04598922]
 [0.48153732]
 [0.43249457]
 [0.2379681 ]
 [0.06579784]]
Autograd ArrayBox with value [[ 0.00483136]
 [ 0.1524863 ]
 [-0.127976

Autograd ArrayBox with value [[ 0.24987508]
 [ 0.42774933]
 [ 0.22211726]
 [ 0.42807085]
 [-0.04445125]]
Autograd ArrayBox with value [[ 0.10614234]
 [ 0.13002754]
 [-0.19884559]
 [-0.08203673]
 [ 0.06427154]]
Autograd ArrayBox with value [[ 0.34522695]
 [ 0.41179098]
 [ 0.13030851]
 [ 0.515777  ]
 [-0.09107139]]
Autograd ArrayBox with value [[ 0.1561174 ]
 [ 0.11934733]
 [-0.2851595 ]
 [-0.12227991]
 [ 0.0309012 ]]
Autograd ArrayBox with value [[ 0.54268515]
 [ 0.38134726]
 [-0.05778978]
 [ 0.69582734]
 [-0.18568407]]
Autograd ArrayBox with value [[ 0.28536714]
 [ 0.09888096]
 [-0.54176245]
 [-0.34073655]
 [-0.07115836]]
Autograd ArrayBox with value [[ 0.20901277]
 [ 0.43561869]
 [ 0.26215282]
 [ 0.39030781]
 [-0.02390735]]
Autograd ArrayBox with value [[ 0.08600296]
 [ 0.13448139]
 [-0.17096582]
 [-0.07647098]
 [ 0.0791358 ]]
Autograd ArrayBox with value [[0.09558774]
 [0.46974068]
 [0.38152337]
 [0.28337387]
 [0.03985214]]
Autograd ArrayBox with value [[ 0.03132368]
 [ 0.14950412]
 

Autograd ArrayBox with value [[ 0.29263818]
 [ 0.43206629]
 [ 0.18424643]
 [ 0.46738737]
 [-0.06015224]]
Autograd ArrayBox with value [[ 0.13110773]
 [ 0.13037048]
 [-0.23834243]
 [-0.10188696]
 [ 0.05255661]]
Autograd ArrayBox with value [[ 0.49047587]
 [ 0.40380856]
 [-0.00523922]
 [ 0.64853891]
 [-0.15462182]]
Autograd ArrayBox with value [[ 0.25319435]
 [ 0.10904713]
 [-0.46848659]
 [-0.27172686]
 [-0.03590088]]
Autograd ArrayBox with value [[ 0.48388796]
 [ 0.40472605]
 [ 0.0010547 ]
 [ 0.64251078]
 [-0.15148893]]
Autograd ArrayBox with value [[ 0.2484207 ]
 [ 0.1096858 ]
 [-0.4592266 ]
 [-0.26295496]
 [-0.03191822]]
Autograd ArrayBox with value [[0.04822527]
 [0.66852151]
 [0.57209324]
 [0.19350591]
 [0.20790203]]
Autograd ArrayBox with value [[ 0.02840121]
 [ 0.25921177]
 [-0.30592208]
 [-0.39530389]
 [ 0.3696515 ]]
Autograd ArrayBox with value [[ 0.18041348]
 [ 0.45165728]
 [ 0.2941428 ]
 [ 0.36400041]
 [-0.00460468]]
Autograd ArrayBox with value [[ 0.07397464]
 [ 0.14210667]
 

 [-0.14768972]]
Autograd ArrayBox with value [[ 0.27435517]
 [ 0.11685113]
 [-0.4866746 ]
 [-0.30903233]
 [-0.04341767]]
Autograd ArrayBox with value [[0.07327291]
 [0.50456513]
 [0.41935825]
 [0.25566925]
 [0.06885381]]
Autograd ArrayBox with value [[ 0.02358007]
 [ 0.16360497]
 [-0.14529364]
 [-0.13728496]
 [ 0.17068756]]
Autograd ArrayBox with value [[0.07902486]
 [0.75399784]
 [0.59115183]
 [0.19638403]
 [0.23249196]]
Autograd ArrayBox with value [[ 0.08390487]
 [ 0.31376817]
 [-0.40692688]
 [-0.5309241 ]
 [ 0.4370452 ]]
Autograd ArrayBox with value [[ 0.35328057]
 [ 0.44463212]
 [ 0.1343302 ]
 [ 0.51424974]
 [-0.07570706]]
Autograd ArrayBox with value [[ 0.17004833]
 [ 0.13190362]
 [-0.29795718]
 [-0.14780251]
 [ 0.03554892]]
Autograd ArrayBox with value [[0.18539738]
 [0.46890764]
 [0.29724181]
 [0.36116233]
 [0.00459892]]
Autograd ArrayBox with value [[ 0.07978426]
 [ 0.14825844]
 [-0.1677923 ]
 [-0.09662468]
 [ 0.10019112]]
Autograd ArrayBox with value [[0.03011487]
 [0.5557026

In [17]:
g_trial_predict(t,P,x_0)
eigval, eigvec = np.linalg.eigh(A)
print(g_trial_predict(t,P,x_0)[-1])
print(eigvec)

[ 0.52461352  0.43819095 -0.02322701  0.66419343 -0.14538251]
[[-0.11591938  0.36630714 -0.31514082 -0.31481908  0.80867607]
 [-0.03697112 -0.06476226  0.82605531  0.3081914   0.46592883]
 [-0.81391888  0.16545258 -0.18429373  0.52208713 -0.06018593]
 [-0.22777979  0.66454556  0.41248819 -0.46070518 -0.35227801]
 [ 0.52043083  0.6266016  -0.1191982   0.56664166 -0.0350885 ]]


In [6]:
#Forward Euler for Yi et al diff.eq
# Solution from np.linalg.eigh
eigval, eigvec = np.linalg.eigh(A)
#print(eigval)
#print(eigvec[:, -1])


x_t = np.random.randn(np.size(A,0)).reshape(-1,1)
#x_t = x_0.reshape(-1,1)

print(eigvec[:, -1] - ( - x_t/np.sqrt(x_t.T @ x_t)).T)
dt = 0.01
#print(x_t) # initial guess
for i in range(1000):
    x_t += (-x_t + (x_t.T @ x_t * A + (1 - x_t.T @ A @ x_t) * np.identity(5)) @ x_t)*dt
#print(x_t/np.sqrt(x_t.T @ x_t)) # normalized eigenvector
#print(x_t.T @ A @ x_t / (x_t.T @ x_t)) # eigenvalue

print(eigvec[:, -1] - ( - x_t/np.sqrt(x_t.T @ x_t)).T)

[[ 0.68814127 -0.33158652 -0.45025328 -0.62102453 -0.38873154]]
[[-4.02796454e-06  3.83532934e-06  2.32355305e-05 -9.33483986e-06
   1.19491700e-05]]


In [7]:
# Solution from np.linalg.eigh

eigval, eigvec = np.linalg.eigh(A)
print(eigval)
print(eigvec[:, -1])

[-1.56887992 -0.48134608 -0.03749509  0.51368892  1.23063218]
[ 0.80867607  0.46592883 -0.06018593 -0.35227801 -0.0350885 ]
